# **Analysis of the influence of aquaculture pond construction on mangrove forests in Asian countries.**

## Introduction
Almost one third of the world's mangrove forests have been lost due to deforestation in the last 50 years (Alongi, 2002; Barbier, 2014). Along with coastal development, another major cause of global mangrove deforestation is the development of shrimp farms to support a booming export fishing industry (Barbier and Cox, 2004; Hamilton, 2020; Richards and Friess, 2016), with global demand for shrimp continuing to rise (Anderson et al., 2019). A study by Hamilton (2013) found that 51.9% of the original mangrove areas have been deforested between the 1970s and after 2004, with commercial aquaculture accounting for 28% of the total mangrove loss in eight countries: Indonesia, Brazil, Bangladesh, India, Thailand, Vietnam, Ecuador and China.

This notebook series studies the influence of aquaculture pond construction on mangrove forests through spatial analysis of two datasets: Aquaculture dynamics (Ottinger et al., 2021) and the Global Mangrove Watch version 3 (Bunting et al., 2018; Bunting, Rosenqvist, Hilarides, Lucas, & Thomas, 2022; Bunting, Rosenqvist, Hilarides, Lucas, Thomas, et al., 2022).

The general objective is to look for a relationship between the activation or construction of aquaculture ponds and the loss of mangrove forest extent. For this purpose, several notebooks were made for the different steps in the analysis process:


1. DataPreparation: The datasets are divided by countries in Asia and the layers are organised in separate folders.
2.   DataAnalysis: We proceed to analyse the relationship between both datasets using geoprocessing tools and working with dataframes.
MangroveChangeAnalysis: Only the Global Mangrove Watch dataset is analysed and an analysis of the dynamics within Asia is made.
4. GraphicOutput: The results of the analysis are visualised for interpretation and understanding.
5.   HexagonMangrove: Mapping the cumulative mangrove forest loss from 1996 to 2020 in the ten countries with the highest mangrove loss in Asia.
6.   HexagonPonds: Mapping cumulative mangrove loss due to pond construction from 1996 to 2019.

# Notebooks 1: Preparation of datasets for further analysis

In this notebook, folders will be created for each country in which future analysis will be carried out. As mentioned above, three datasets will be used:

1.   Global Mangrove Watch version 3
2.   Aquaculture Dynamics
3.   The Open Street Map national boundaries layer.

The OSM boundaries were selected because of the errors and differences in shoreline between the different layers. Thus if mangrove forest is lost in the first coastal strip, for example, it would not be observed due to the use of a fixed shoreline to trim the layers.



We start by importing the necessary python libraries.

In [1]:
# Import necessary libraries
import geopandas as gpd
import os

Then we define the variables to be used in this case are only the years in which we are going to do our analysis. These years are the ones that coincide both data sets. We also define the address of the files to be used.

In [5]:
#Define the years for analysis
years = [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019]

# File paths
ponds_path = 'Aquaculture_Dynamics_Asia/asia_aquaculture_ponds_dynamics_stats_otsu_1984_to_2019.gpkg'
boundaries_path = 'OSMAsiaBoundariesL2.gpkg'
mangroves_paths = [f'GMW_v3/asia/gmw_v3_asia_{year}.shp' for year in years]
hexagon_path = 'hexagons_output.gpkg'

# Load pond layer
ponds = gpd.read_file(ponds_path)

# Load country boundaries layer
boundaries = gpd.read_file(boundaries_path)

# Load mangrove layers
mangroves = [gpd.read_file(path) for path in mangroves_paths]

# Load hexagon layers
hexagon = gpd.read_file(hexagon_pathds)

Then we define the countries where we want to do our analysis and create the folders where we are going to store the cut and ready layers for each country. In this case we will analyse the first 10 countries where mangrove loss has occurred in Asia region.

In [6]:
# Define the list of countries for which folders will be created
countries = ["Indonesia", "Myanmar", "India", "Philippines", "Vietnam", "Bangladesh", "Malaysia", "Thailand", "Pakistan", "China"]

# Create a folder for each country if it does not exist
for country in countries:
    os.makedirs(country, exist_ok=True)


Finally, we create a loop for each country in order to make Clip, Transform, and Save the Layers in each folder. We use for our analysis the projection WGS 84  Pseudo-Mercator, Spherical Mercator used by Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI

In [7]:
# Iterate through the countries
for country in countries:
    # Get the boundary for the specific country
    boundary = boundaries[boundaries['name'] == country]
    
    
    clipped_hex = gpd.clip(hexagon, boundary)

    # Clip the pond layer using the country boundary
    clipped_ponds = gpd.clip(ponds, boundary)

    # Clip the mangrove layers using the country boundary
    clipped_mangroves = [gpd.clip(mangrove, boundary) for mangrove in mangroves]

    # Convert the clipped hexagon layer to EPSG:3857 and save as GPKG
    clipped_hex.to_crs(epsg=3857).to_file(os.path.join(country, '_hex.gpkg'), driver='GPKG')
    
    # Convert the clipped pond layer to EPSG:3857 and save as GPKG
    #lipped_ponds.to_crs(epsg=3857).to_file(os.path.join(country, '_ponds.gpkg'), driver='GPKG')
    
    # Convert the clipped mangrove layers to EPSG:3857 and save as GPKG
    for year, mangrove in zip(years, clipped_mangroves):  # Corrected years
        mangrove.to_crs(epsg=3857).to_file(os.path.join(country, f'_mangroves_{year}.gpkg'), driver='GPKG')



Within each country you will find the following folders